In [ ]:
def initial_u_matrix(0):
    """
    Creates a very specific matrix that includes all our initial and boundary values such that our
    unknown values are 0.
    
    This foundation will be:
    [u(x_0,t_0) ; u(x_0,t_1) ; u(x_0,t_2) ; ... ; u(x_0,t_n)]
    [u(x_1,t_0) ;     0      ;     0      ; ... ;     0     ]
    [u(x_2,t_0) ;     0      ;     0      ; ... ;     0     ]
    [    ...    ;     0      ;     0      ; ... ;     0     ]
    [u(x_n,t_0) ; u(x_n,t_1) ; u(x_n,t_2) ; ... ; u(x_n,t_n)]
    =
    [ x_0-x_n ;  x_n+t_1  ;  x_n+t_2  ; ... ;  x_n+t_n  ]
    [ x_1-x_n ;     0     ;     0     ; ... ;     0     ]
    [ x_2-x_n ;     0     ;     0     ; ... ;     0     ]
    [   ...   ;     0     ;     0     ; ... ;     0     ]
    [    0    ;     0     ;     0     ; ... ;     0     ]
    """
    import numpy as np
    
    # Create our zero matrix and make the first row 1
    matrix = np.zeros((x_matrix.shape))
    matrix[0] = 1
    
    # Make the first column e^x and last row e^t
    matrix[:,0] = np.exp(x_matrix[:,0])
    matrix[-1] = np.exp(t_matrix[-1])
    
    return matrix